In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1zd_JQpXU6AEi_x1xeTp1ryM4G2k59LVE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hymenoptera_data.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "hymenoptera_data.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [9]:
cd /content

/content


*モデルの適用*

In [10]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 2.332s
     |████████████████████████████████| 686kB 3.4MB/s 
     |████████████████████████████████| 204kB 63.8MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=0c57e333284db733c0074198534074861084f523ca8b4738891bef52c8413141
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=aabd30691e327be0a41c0c43b0a277a12124e04beaf26cb5fe22efc65ec05b04
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 115 ms, sys: 34.9 ms, total: 150 ms
Wall time: 16.6 s


In [11]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 9.74 ms, sys: 8.1 ms, total: 17.8 ms
Wall time: 6.18 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [13]:
%%time
net = network.VGG16(class_size=2)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/train', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/val', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 3.08 ms, sys: 17 µs, total: 3.1 ms
Wall time: 5.29 ms


In [15]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.96 ms, sys: 33.8 ms, total: 36.8 ms
Wall time: 58.5 ms


In [16]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-23 11:52:06.306220
val Loss: 0.6921 Acc: 0.5425
----------
Epoch 2/50 2020-01-23 11:52:08.483703
train Loss: 976.0234 Acc: 0.1230
val Loss: 0.6959 Acc: 0.4575
----------
Epoch 3/50 2020-01-23 11:52:16.090750
train Loss: 0.6986 Acc: 0.5041
val Loss: 0.6915 Acc: 0.5425
----------
Epoch 4/50 2020-01-23 11:52:23.647563
train Loss: 0.6965 Acc: 0.4959
val Loss: 0.6886 Acc: 0.5425
----------
Epoch 5/50 2020-01-23 11:52:31.310692
train Loss: 0.7547 Acc: 0.4959
val Loss: 0.6924 Acc: 0.5425
----------
Epoch 6/50 2020-01-23 11:52:38.998883
train Loss: 0.6941 Acc: 0.4959
val Loss: 0.6923 Acc: 0.5425
----------
Epoch 7/50 2020-01-23 11:52:46.677812
train Loss: 0.6946 Acc: 0.4959
val Loss: 0.6925 Acc: 0.5425
----------
Epoch 8/50 2020-01-23 11:52:54.304662
train Loss: 0.6933 Acc: 0.4959
val Loss: 0.6925 Acc: 0.5425
----------
Epoch 9/50 2020-01-23 11:53:01.956618
train Loss: 0.6933 Acc: 0.4959
val Loss: 0.6925 Acc: 0.5425
----------
Epoch 10/50 2020-01-23 11:53:09.50247

In [17]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 54 %
CPU times: user 486 ms, sys: 403 ms, total: 889 ms
Wall time: 2.25 s
